pip install faiss-cpu

In [2]:
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---- ----------------------------------- 2.1/18.2 MB 16.7 MB/s eta 0:00:01
   ---------------- ----------------------- 7.3/18.2 MB 21.6 MB/s eta 0:00:01
   --------------------------- ------------ 12.3/18.2 MB 22.0 MB/s eta 0:00:01
   -------------------------------------- - 17.3/18.2 MB 23.2 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 19.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Từ Multi Labels Classification => Hierarchical Classification sử dụng FAISS

In [5]:
# Ý TƯỞNG HIERARCHICAL CLASSIFICATION - GIẢI THÍCH CHI TIẾT

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

print("="*60)
print("🎯 Ý TƯỞNG HIERARCHICAL CLASSIFICATION")
print("="*60)

# =============================================================================
# 1. VẤN ĐỀ: FLAT CLASSIFICATION vs HIERARCHICAL CLASSIFICATION
# =============================================================================

print("\n💡 VẤN ĐỀ BAN ĐẦU:")
print("""
Giả sử bạn cần phân loại sinh viên vào các lĩnh vực:
Math, Physics, Chemistry, Biology, Statistics, Literature, History, 
Art, Design, Economics, Finance, CS, AI

🤔 CÓ 2 CÁCH TIẾP CẬN:

❌ FLAT CLASSIFICATION (cách thông thường):
   Input → Classifier → [Math, Physics, Chemistry, Biology, ...]
   → Một classifier duy nhất phải học 13 classes!

✅ HIERARCHICAL CLASSIFICATION (cách thông minh):
   Input → Parent Classifier → [Science, Arts, Business, ComputerScience]
         → Child Classifiers → [Math, Physics...] (cho Science)
                            → [Literature, History...] (cho Arts)
                            → [Economics, Finance] (cho Business)  
                            → [CS, AI] (cho ComputerScience)
""")

# =============================================================================
# 2. MINH HỌA BẰNG VÍ DỤ CỤ THỂ
# =============================================================================

print("\n🔍 MINH HỌA BẰNG VÍ DỤ:")

# Dữ liệu mẫu
X_texts = [
    "student likes math and programming",
    "student loves physics and chemistry", 
    "student enjoys literature and history",
    "student interested in biology and chemistry",
    "student good at math and statistics",
    "student passionate about computer science and ai",
    "student enjoys economics and finance",  # Thêm sample Business
    "student loves art and design"          # Thêm sample Arts khác
]

y_children = [
    ["Math", "CS"],
    ["Physics", "Chemistry"],
    ["Literature", "History"],
    ["Biology", "Chemistry"], 
    ["Math", "Statistics"],
    ["CS", "AI"],
    ["Economics", "Finance"],  # Thêm Business labels
    ["Art", "Design"]          # Thêm Arts labels khác
]

print("📚 Training Data:")
for i, (text, labels) in enumerate(zip(X_texts, y_children)):
    print(f"  {i+1}. '{text}' → {labels}")

# Mapping children → parents
child2parent = {
    "Math": "Science", "Physics": "Science", "Chemistry": "Science",
    "Biology": "Science", "Statistics": "Science",
    "Literature": "Arts", "History": "Arts", "Art": "Arts", "Design": "Arts",
    "Economics": "Business", "Finance": "Business",
    "CS": "ComputerScience", "AI": "ComputerScience"
}

print(f"\n🌳 HIERARCHY STRUCTURE:")
parent_to_children = {}
for child, parent in child2parent.items():
    parent_to_children.setdefault(parent, []).append(child)

for parent, children in parent_to_children.items():
    print(f"  {parent:15s} → {children}")

# =============================================================================
# 3. BƯỚC 1: TỪ CHILDREN LABELS → PARENT LABELS
# =============================================================================

print(f"\n📊 BƯỚC 1: Derive Parent Labels từ Children Labels")
y_parents = []
for i, childs in enumerate(y_children):
    parents = sorted({child2parent[c] for c in childs})
    y_parents.append(parents)
    print(f"  Sample {i+1}: {childs} → Parents: {parents}")

print(f"\nKết quả:")
print(f"  Children labels: {y_children}")
print(f"  Parent labels:   {y_parents}")

# =============================================================================
# 4. BƯỚC 2: TRAIN 2-STAGE CLASSIFIERS
# =============================================================================

print(f"\n🤖 BƯỚC 2: Train Classifiers")

# Encode labels
mlb_parents = MultiLabelBinarizer()
Y_parents = mlb_parents.fit_transform(y_parents)
parent_names = list(mlb_parents.classes_)

mlb_children = MultiLabelBinarizer()
Y_children = mlb_children.fit_transform(y_children)
child_names = list(mlb_children.classes_)

print(f"Parent classes: {parent_names}")
print(f"Child classes:  {child_names}")

# Vectorize
vectorizer = TfidfVectorizer(max_features=100)
X_vec = vectorizer.fit_transform(X_texts).toarray()

print(f"\nTF-IDF vectors shape: {X_vec.shape}")

# Train parent classifier
parent_clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
parent_clf.fit(X_vec, Y_parents)
print(f"✅ Parent classifier trained!")

# Train child classifiers cho từng parent
child_name_to_idx = {name: i for i, name in enumerate(child_names)}
parent_child_indices = {
    p: [child_name_to_idx[c] for c in parent_to_children[p]] 
    for p in parent_to_children
}

child_clfs = {}
print(f"\n🔧 Training Child Classifiers:")
for i, parent in enumerate(parent_names):
    # Tìm samples có parent này
    idx_pos = np.where(Y_parents[:, i] == 1)[0]
    print(f"  Parent '{parent}': {len(idx_pos)} samples")
    
    if len(idx_pos) < 2:
        child_clfs[parent] = None
        print(f"    ⚠️  Không đủ data → Skip")
        continue
    
    # Lấy subset data cho parent này
    X_sub = X_vec[idx_pos]
    cols = parent_child_indices[parent]  # child indices cho parent này
    y_sub = Y_children[idx_pos][:, cols]  # chỉ lấy children của parent này
    
    print(f"    Training data shape: {X_sub.shape}, Labels shape: {y_sub.shape}")
    print(f"    Children: {[child_names[c] for c in cols]}")
    
    # Train classifier cho children của parent này
    clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
    clf.fit(X_sub, y_sub)
    child_clfs[parent] = (clf, cols)
    print(f"    ✅ Trained!")

# =============================================================================
# 5. BƯỚC 3: PREDICTION WORKFLOW
# =============================================================================

print(f"\n🔮 BƯỚC 3: Prediction Workflow")

def predict_step_by_step(text):
    print(f"\n📝 Query: '{text}'")
    print("-" * 40)
    
    # Vectorize input
    query_vec = vectorizer.transform([text]).toarray()
    print(f"1️⃣ Vectorized input: {query_vec.shape}")
    
    # Stage 1: Predict parents
    parent_probs = parent_clf.predict_proba(query_vec).reshape(-1)
    print(f"2️⃣ Parent predictions:")
    threshold = 0.3
    active_parents = []
    for i, (parent, prob) in enumerate(zip(parent_names, parent_probs)):
        status = "✅ ACTIVE" if prob >= threshold else "❌ inactive"
        print(f"   {parent:15s}: {prob:.3f} {status}")
        if prob >= threshold:
            active_parents.append((parent, prob))
    
    # Stage 2: Predict children cho mỗi active parent
    print(f"3️⃣ Child predictions for active parents:")
    all_predicted_children = []
    
    for parent_name, parent_prob in active_parents:
        print(f"   📂 Parent: {parent_name}")
        clf_info = child_clfs.get(parent_name)
        
        if clf_info is None:
            print(f"      ⚠️  No child classifier trained")
            continue
            
        clf, cols = clf_info
        child_probs = clf.predict_proba(query_vec)
        child_probs = np.array(child_probs).reshape(-1)
        
        child_threshold = 0.4
        for j, child_col_idx in enumerate(cols):
            child_name = child_names[child_col_idx]
            prob = child_probs[j]
            status = "✅" if prob >= child_threshold else "❌"
            print(f"      {child_name:12s}: {prob:.3f} {status}")
            if prob >= child_threshold:
                all_predicted_children.append(child_name)
    
    print(f"🎯 FINAL RESULT: {sorted(set(all_predicted_children))}")
    return sorted(set(all_predicted_children))

# Test với các query khác nhau
test_queries = [
    "student loves mathematics and programming",
    "student enjoys reading novels and poetry", 
    "student interested in business and finance",
    "student passionate about physics and chemistry",
    "student loves drawing and creative design"
]

for query in test_queries:
    predict_step_by_step(query)

# =============================================================================
# 6. TẠI SAO HIERARCHICAL TỐT HỢN?
# =============================================================================

print(f"\n\n🚀 TẠI SAO HIERARCHICAL CLASSIFICATION TỐT HỢN?")
print("""
✅ **ƯU ĐIỂM:**

1️⃣ **Giảm complexity**: 
   - Thay vì 1 classifier học 13 classes
   - → 1 parent classifier (4 classes) + 4 child classifiers (2-5 classes mỗi cái)

2️⃣ **Tăng accuracy**:
   - Parent classifier tập trung vào high-level concepts
   - Child classifiers tập trung vào fine-grained distinctions

3️⃣ **Handle imbalanced data tốt hơn**:
   - Mỗi child classifier chỉ cần phân biệt trong phạm vi parent
   - Ít bị nhiễu từ classes không liên quan

4️⃣ **Interpretable**:
   - Biết được tại sao predict class này (qua parent reasoning)
   - Dễ debug và tune

5️⃣ **Scalable**:
   - Có thể thêm parents/children mới dễ dàng
   - Không cần retrain toàn bộ system

❌ **NHƯỢC ĐIỂM:**
- Phức tạp hơn để implement
- Error propagation (sai parent → sai children)
- Cần define hierarchy structure
""")

# =============================================================================
# 7. VAI TRÒ CỦA FAISS/KNN
# =============================================================================

print(f"\n🔍 VAI TRÒ CỦA FAISS/NEIGHBORS:")
print("""
🤔 **TẠI SAO CẦN NEIGHBORS?**

Classifier đôi khi miss patterns → Neighbors giúp "gợi ý":

1️⃣ **Ensemble prediction**:
   final_prob = α × classifier_prob + (1-α) × neighbor_prob

2️⃣ **Handle edge cases**:
   - Input không giống training data
   - Neighbors cung cấp "similar examples"

3️⃣ **Improve recall**:
   - Classifier có thể quá conservative
   - Neighbors giúp "bổ sung" labels bị miss

VÍ DỤ:
Query: "student loves calculus and neural networks"
→ Classifier: Science(0.6), ComputerScience(0.3) 
→ Neighbors: có samples về Math+AI → boost ComputerScience
→ Final: Science(0.5), ComputerScience(0.7) ← CẢ HAI đều active!
""")

print(f"\n💡 **TÓM LẠI:**")
print("Hierarchical Classification = Chia để trị + Ensemble prediction")
print("→ Accurate hơn, Scalable hơn, Interpretable hơn flat classification!")

🎯 Ý TƯỞNG HIERARCHICAL CLASSIFICATION

💡 VẤN ĐỀ BAN ĐẦU:

Giả sử bạn cần phân loại sinh viên vào các lĩnh vực:
Math, Physics, Chemistry, Biology, Statistics, Literature, History, 
Art, Design, Economics, Finance, CS, AI

🤔 CÓ 2 CÁCH TIẾP CẬN:

❌ FLAT CLASSIFICATION (cách thông thường):
   Input → Classifier → [Math, Physics, Chemistry, Biology, ...]
   → Một classifier duy nhất phải học 13 classes!

✅ HIERARCHICAL CLASSIFICATION (cách thông minh):
   Input → Parent Classifier → [Science, Arts, Business, ComputerScience]
         → Child Classifiers → [Math, Physics...] (cho Science)
                            → [Literature, History...] (cho Arts)
                            → [Economics, Finance] (cho Business)  
                            → [CS, AI] (cho ComputerScience)


🔍 MINH HỌA BẰNG VÍ DỤ:
📚 Training Data:
  1. 'student likes math and programming' → ['Math', 'CS']
  2. 'student loves physics and chemistry' → ['Physics', 'Chemistry']
  3. 'student enjoys literature and history'

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\multiclass.py:90: UserWarning: Label 0 is present in all training examples.
  warnings.warn(


# Có thư viện nào hỗ trợ ko 

In [ ]:
# Thư viện Hierarchical Classification có sẵn - So sánh và Demo

"""
🚀 TOP 3 THƯ VIỆN HIERARCHICAL CLASSIFICATION

1. HiClass (KHUYẾN NGHỊ) - Mới nhất, active development
2. sklearn-hierarchical-classification - Stable, nhiều tính năng
3. scikit-multilearn - Tập trung multi-label
"""

print("="*60)
print("📚 THƯ VIỆN HIERARCHICAL CLASSIFICATION CÓ SẴN")
print("="*60)

# =============================================================================
# 1. HiClass - THƯ VIỆN CHÍNH (KHUYẾN NGHỊ)
# =============================================================================

print("""
🥇 **1. HiClass - KHUYẾN NGHỊ SỬ DỤNG**

📦 **Installation:**
pip install hiclass
# hoặc
conda install hiclass

🌟 **Ưu điểm:**
✅ Compatible hoàn toàn với scikit-learn
✅ Active development (2021-2025)
✅ Documentation đầy đủ với interactive notebooks
✅ Hỗ trợ parallel training
✅ Built-in hierarchical metrics
✅ 3 phương pháp chính: LCPN, LCPPN, LCPL

🎯 **Phương pháp hỗ trợ:**
- LocalClassifierPerNode (LCPN)
- LocalClassifierPerParentNode (LCPPN) 
- LocalClassifierPerLevel (LCPL)
""")

# Demo HiClass usage
print("\n🔧 **HiClass DEMO:**")

hiclass_demo = '''
from hiclass import LocalClassifierPerParentNode
from sklearn.ensemble import RandomForestClassifier

# Sample data
X_train = [
    "student likes math and programming",
    "student loves physics and chemistry", 
    "student enjoys literature and history"
]

# Hierarchical labels (mỗi cột = 1 level)
y_train = [
    ["Science", "Math"],        # Level 0: Science, Level 1: Math
    ["Science", "Physics"],     # Level 0: Science, Level 1: Physics  
    ["Arts", "Literature"]      # Level 0: Arts, Level 1: Literature
]

# Initialize và train
classifier = LocalClassifierPerParentNode(
    local_classifier=RandomForestClassifier()
)
classifier.fit(X_train, y_train)

# Predict
X_test = ["student passionate about computer science"]
predictions = classifier.predict(X_test)
print("Prediction:", predictions)

# Get probabilities
probabilities = classifier.predict_proba(X_test)
print("Probabilities:", probabilities)
'''

print(hiclass_demo)

# =============================================================================
# 2. sklearn-hierarchical-classification
# =============================================================================

print("""
🥈 **2. sklearn-hierarchical-classification**

📦 **Installation:**
pip install sklearn-hierarchical-classification

🌟 **Ưu điểm:**
✅ Mature, stable codebase
✅ Rich documentation với examples
✅ Hỗ trợ nhiều evaluation metrics
✅ Interactive development support (tqdm progress bars)
✅ Flexible class hierarchy definition

⚠️ **Nhược điểm:**
- Ít được maintain gần đây
- API phức tạp hơn HiClass
""")

sklearn_hierarchical_demo = '''
from sklearn_hierarchical_classification.classifier import HierarchicalClassifier
from sklearn.svm import LinearSVC

# Define class hierarchy
class_hierarchy = {
    "Root": {
        "Science": ["Math", "Physics", "Chemistry"],
        "Arts": ["Literature", "History", "Art"]
    }
}

# Initialize
classifier = HierarchicalClassifier(
    base_estimator=LinearSVC(),
    class_hierarchy=class_hierarchy
)

# Fit và predict tương tự sklearn
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
'''

print("🔧 **sklearn-hierarchical DEMO:**")
print(sklearn_hierarchical_demo)

# =============================================================================
# 3. scikit-multilearn
# =============================================================================

print("""
🥉 **3. scikit-multilearn**

📦 **Installation:**
pip install scikit-multilearn

🌟 **Ưu điểm:**
✅ Comprehensive multi-label library
✅ Có hierarchical classification module
✅ Many evaluation metrics
✅ Good integration với scikit-learn

⚠️ **Nhược điểm:**
- Hierarchical features ít hơn 2 thư viện trên
- Tập trung chính vào multi-label, hierarchical là addon
""")

# =============================================================================
# 4. SO SÁNH CHI TIẾT
# =============================================================================

print("""
📊 **SO SÁNH CHI TIẾT:**

| Aspect | HiClass | sklearn-hierarchical | scikit-multilearn |
|--------|---------|---------------------|-------------------|
| **Ease of use** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | ⭐⭐⭐ |
| **Documentation** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **Active development** | ⭐⭐⭐⭐⭐ | ⭐⭐ | ⭐⭐⭐ |
| **Hierarchical focus** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐ |
| **Performance** | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐ |
| **Sklearn compatible** | ⭐⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ |

🏆 **WINNER: HiClass**
""")

# =============================================================================
# 5. HiClass COMPLETE EXAMPLE
# =============================================================================

print("""
🎯 **HiClass COMPLETE EXAMPLE - THAY THẾ CODE CỦA BẠN:**
""")

complete_example = '''
# Cài đặt
# pip install hiclass

import numpy as np
from hiclass import LocalClassifierPerParentNode
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

# Data chuẩn bị cho HiClass
X_texts = [
    "student likes math and programming",
    "student loves physics and chemistry", 
    "student enjoys literature and history",
    "student interested in biology and chemistry",
    "student good at math and statistics",
    "student passionate about computer science and ai",
    "student enjoys economics and finance",
    "student loves art and design"
]

# HiClass expects matrix format: [level0, level1, level2, ...]
y_hierarchical = [
    ["Science", "Math"],
    ["Science", "Physics"], 
    ["Arts", "Literature"],
    ["Science", "Biology"],
    ["Science", "Math"],
    ["ComputerScience", "AI"],
    ["Business", "Economics"],
    ["Arts", "Art"]
]

# Vectorize (HiClass làm việc với vectors, không phải raw text)
vectorizer = TfidfVectorizer(max_features=1000)
X_vectors = vectorizer.fit_transform(X_texts).toarray()

# 1. LCPPN - Giống code của bạn
lcppn = LocalClassifierPerParentNode(
    local_classifier=RandomForestClassifier(n_estimators=10)
)
lcppn.fit(X_vectors, y_hierarchical)

# 2. Predict
test_texts = [
    "student loves mathematics and programming",
    "student enjoys reading novels and poetry"
]
test_vectors = vectorizer.transform(test_texts).toarray()

predictions = lcppn.predict(test_vectors)
probabilities = lcppn.predict_proba(test_vectors)

print("Predictions:", predictions)
print("Probabilities:", probabilities)

# 3. Evaluate with hierarchical metrics
from hiclass import hmeasures

# Hierarchical precision, recall, f1
h_precision = hmeasures.precision(y_true, y_pred)
h_recall = hmeasures.recall(y_true, y_pred)
h_f1 = hmeasures.f1(y_true, y_pred)

print(f"H-Precision: {h_precision:.3f}")
print(f"H-Recall: {h_recall:.3f}")
print(f"H-F1: {h_f1:.3f}")
'''

print(complete_example)

# =============================================================================
# 6. KẾT LUẬN VÀ KHUYẾN NGHỊ
# =============================================================================

print("""
💡 **KẾT LUẬN & KHUYẾN NGHỊ:**

🎯 **Nên dùng HiClass vì:**

1️⃣ **Dễ dùng nhất**: API đơn giản, tương tự sklearn
2️⃣ **Active development**: Được maintain tích cực
3️⃣ **Full-featured**: Đầy đủ tính năng hierarchical classification
4️⃣ **Good documentation**: Examples, tutorials đầy đủ
5️⃣ **Performance**: Optimized và hỗ trợ parallel training

🔄 **Migration từ code của bạn:**
- Thay thế toàn bộ implementation
- Chỉ cần format lại data theo matrix format
- Giảm code từ 200+ lines → 10-15 lines
- Tăng performance và reliability

📈 **Lợi ích:**
✅ Tiết kiệm thời gian development
✅ Tăng độ tin cậy (tested extensively)
✅ Dễ maintain và scale
✅ Built-in evaluation metrics
✅ Professional-grade implementation

❌ **Không cần tự implement nữa!**
""")

print("""
🚀 **NEXT STEPS:**

1. `pip install hiclass`
2. Convert data sang matrix format
3. Replace toàn bộ code với HiClass
4. Test performance với hierarchical metrics
5. Deploy production!

HiClass = Thay thế hoàn toàn code hierarchical classification tự viết! 🎉
""")

# Sử dụng Hiclass - demo

In [7]:
!pip install hiclass

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------ --------- 1.6/2.0 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 8.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
# HiClass Simple Demo - Hierarchical Classification
# Thay thế hoàn toàn code tự implement

# Cài đặt: pip install hiclass

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# HiClass imports
from hiclass import LocalClassifierPerParentNode, LocalClassifierPerNode, LocalClassifierPerLevel

print("=" * 60)
print("HiClass Demo - Hierarchical Classification")
print("=" * 60)

# =============================================================================
# 1. CHUẨN BỊ DỮ LIỆU
# =============================================================================

print("\n1. Chuẩn bị dữ liệu:")

# Dữ liệu text
X_texts = [
    "student likes math and programming",
    "student loves physics and chemistry", 
    "student enjoys literature and history",
    "student interested in biology and chemistry",
    "student good at math and statistics",
    "student passionate about computer science and ai",
    "student enjoys economics and finance",
    "student loves art and design",
    "student studies calculus and algorithms", 
    "student reads novels and poetry",
    "student analyzes market trends",
    "student creates digital artwork"
]

# HiClass format: mỗi sample = [level0, level1, level2, ...]
# Khác với format tự implement (list of lists cho multi-label)
y_hierarchical = [
    ["Science", "Math"],              # Level 0: Science, Level 1: Math
    ["Science", "Physics"],           # Level 0: Science, Level 1: Physics
    ["Arts", "Literature"],           # Level 0: Arts, Level 1: Literature
    ["Science", "Biology"],           # Level 0: Science, Level 1: Biology
    ["Science", "Statistics"],        # Level 0: Science, Level 1: Statistics
    ["ComputerScience", "AI"],        # Level 0: CS, Level 1: AI
    ["Business", "Economics"],        # Level 0: Business, Level 1: Economics
    ["Arts", "Design"],               # Level 0: Arts, Level 1: Design
    ["Science", "Math"],              # Thêm samples
    ["Arts", "Literature"],
    ["Business", "Finance"],
    ["Arts", "Design"]
]

print(f"Training samples: {len(X_texts)}")
print("Sample data:")
for i in range(3):
    print(f"  '{X_texts[i]}' -> {y_hierarchical[i]}")

# =============================================================================
# 2. VECTOR HÓA DỮ LIỆU
# =============================================================================

print("\n2. Vector hóa:")

# HiClass cần vectors, không phải raw text
vectorizer = TfidfVectorizer(max_features=500)
X_vectors = vectorizer.fit_transform(X_texts).toarray()

print(f"Vector shape: {X_vectors.shape}")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y_hierarchical, test_size=0.3, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# =============================================================================
# 3. HiClass MODELS - 3 PHƯƠNG PHÁP CHÍNH
# =============================================================================

print("\n3. HiClass Models:")

# Method 1: Local Classifier Per Parent Node (LCPPN) - Giống code bạn tự viết
print("\n   3.1 LocalClassifierPerParentNode (LCPPN):")
lcppn = LocalClassifierPerParentNode(
    local_classifier=RandomForestClassifier(n_estimators=10, random_state=42)
)
lcppn.fit(X_train, y_train)
print("   ✅ LCPPN trained")

# Method 2: Local Classifier Per Node (LCPN)
print("\n   3.2 LocalClassifierPerNode (LCPN):")
lcpn = LocalClassifierPerNode(
    local_classifier=LogisticRegression(max_iter=1000, random_state=42)
)
lcpn.fit(X_train, y_train)
print("   ✅ LCPN trained")

# Method 3: Local Classifier Per Level (LCPL) 
print("\n   3.3 LocalClassifierPerLevel (LCPL):")
lcpl = LocalClassifierPerLevel(
    local_classifier=RandomForestClassifier(n_estimators=10, random_state=42)
)
lcpl.fit(X_train, y_train)
print("   ✅ LCPL trained")

# =============================================================================
# 4. PREDICTION
# =============================================================================

print("\n4. Predictions:")

# Test với các query mới
test_texts = [
    "student loves mathematics and programming", 
    "student enjoys reading novels and poetry",
    "student interested in business analysis"
]

# Vector hóa test data
test_vectors = vectorizer.transform(test_texts).toarray()

print("\nPrediction results:")
for i, text in enumerate(test_texts):
    print(f"\nQuery: '{text}'")
    
    # LCPPN predictions
    pred_lcppn = lcppn.predict([test_vectors[i]])[0]
    print(f"  LCPPN: {pred_lcppn}")
    
    # LCPN predictions  
    pred_lcpn = lcpn.predict([test_vectors[i]])[0]
    print(f"  LCPN:  {pred_lcpn}")
    
    # LCPL predictions
    pred_lcpl = lcpl.predict([test_vectors[i]])[0]
    print(f"  LCPL:  {pred_lcpl}")

# =============================================================================
# 5. PROBABILITIES
# =============================================================================

print("\n5. Probabilities:")

# Lấy probabilities từ LCPPN
probabilities = lcppn.predict_proba(test_vectors)

print("\nLCPPN Probabilities:")
for i, text in enumerate(test_texts):
    print(f"\nQuery: '{text}'")
    prob = probabilities[i]
    print(f"  Probabilities: {prob}")

# =============================================================================
# 6. HIERARCHICAL EVALUATION METRICS
# =============================================================================

print("\n6. Hierarchical Evaluation:")

# HiClass có built-in hierarchical metrics
try:
    from hiclass.metrics import precision, recall, f1
    
    # Predict trên test set
    y_pred = lcppn.predict(X_test)
    
    # Calculate hierarchical metrics
    h_precision = precision(y_test, y_pred)
    h_recall = recall(y_test, y_pred)
    h_f1 = f1(y_test, y_pred)
    
    print(f"Hierarchical Precision: {h_precision:.3f}")
    print(f"Hierarchical Recall: {h_recall:.3f}")
    print(f"Hierarchical F1: {h_f1:.3f}")

except ImportError:
    print("Hierarchical metrics require newer version of hiclass")
    
    # Alternative: manual evaluation
    correct = 0
    total = len(y_test)
    
    for true, pred in zip(y_test, y_pred):
        if true == pred:
            correct += 1
    
    accuracy = correct / total
    print(f"Simple Accuracy: {accuracy:.3f}")

# =============================================================================
# 7. SO SÁNH VỚI IMPLEMENTATION TỰ VIẾT
# =============================================================================

print(f"\n7. So sánh với implementation tự viết:")

print("""
📊 COMPARISON:

Tự implement:                    HiClass:
==============                   =========
✏️  200+ lines code             ✅ 10-15 lines code
⏰ Vài tuần development         ✅ Vài giờ setup  
🐛 Tự handle bugs              ✅ Tested extensively
⚙️  Manual optimization        ✅ Optimized sẵn
📚 Tự viết documentation       ✅ Full documentation
🔄 Tự implement metrics        ✅ Built-in hierarchical metrics
🛠️  Manual parallel support    ✅ Auto parallel training
💾 Tự implement save/load      ✅ Pickle support sẵn

🎯 RESULT: HiClass tiết kiệm 90% effort!
""")

# =============================================================================
# 8. SAVE/LOAD MODEL
# =============================================================================

print("\n8. Save/Load Model:")

# Save model (pickle support sẵn)
import pickle

# Save
with open('hiclass_model.pkl', 'wb') as f:
    pickle.dump(lcppn, f)
print("✅ Model saved")

# Load
with open('hiclass_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
print("✅ Model loaded")

# Test loaded model
test_pred = loaded_model.predict([test_vectors[0]])[0]
print(f"Loaded model prediction: {test_pred}")

# =============================================================================
# 9. ADVANCED FEATURES
# =============================================================================

print("\n9. Advanced Features:")

print("""
🚀 ADVANCED HiClass FEATURES:

1️⃣ Custom Base Classifiers:
   - RandomForest, SVM, LogisticRegression
   - XGBoost, LightGBM compatibility
   - Any sklearn-compatible estimator

2️⃣ Parallel Training:
   - Auto parallel training across hierarchy levels
   - Faster training trên multi-core systems

3️⃣ Sparse Matrix Support:
   - Memory efficient cho large datasets
   - CSR, CSC matrix support

4️⃣ Missing Labels Handling:
   - Auto handle incomplete hierarchical paths
   - Robust với noisy data

5️⃣ Custom Hierarchy Validation:
   - Auto validate hierarchy consistency
   - Error detection trong label structure
""")

# =============================================================================
# 10. PRODUCTION USAGE TEMPLATE
# =============================================================================

print("\n10. Production Usage Template:")

production_template = '''
# Production Template
from hiclass import LocalClassifierPerParentNode
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

class HierarchicalTextClassifier:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(max_features=5000)
        self.classifier = LocalClassifierPerParentNode(
            local_classifier=RandomForestClassifier(n_estimators=100)
        )
    
    def fit(self, texts, labels):
        X = self.vectorizer.fit_transform(texts).toarray()
        self.classifier.fit(X, labels)
    
    def predict(self, texts):
        X = self.vectorizer.transform(texts).toarray()
        return self.classifier.predict(X)
    
    def save(self, filepath):
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)
    
    @classmethod
    def load(cls, filepath):
        with open(filepath, 'rb') as f:
            return pickle.load(f)

# Usage
classifier = HierarchicalTextClassifier()
classifier.fit(X_texts, y_hierarchical)
predictions = classifier.predict(["new text to classify"])
'''

print(production_template)

print(f"\n🎉 HiClass Demo completed!")
print("Next steps: pip install hiclass và thay thế code tự viết!")

HiClass Demo - Hierarchical Classification

1. Chuẩn bị dữ liệu:
Training samples: 12
Sample data:
  'student likes math and programming' -> ['Science', 'Math']
  'student loves physics and chemistry' -> ['Science', 'Physics']
  'student enjoys literature and history' -> ['Arts', 'Literature']

2. Vector hóa:
Vector shape: (12, 38)
Train: (8, 38), Test: (4, 38)

3. HiClass Models:

   3.1 LocalClassifierPerParentNode (LCPPN):
   ✅ LCPPN trained

   3.2 LocalClassifierPerNode (LCPN):
   ✅ LCPN trained

   3.3 LocalClassifierPerLevel (LCPL):
   ✅ LCPL trained

4. Predictions:

Prediction results:

Query: 'student loves mathematics and programming'
  LCPPN: ['Arts' 'Design']
  LCPN:  ['Arts' 'Design']
  LCPL:  ['Arts' 'Design']

Query: 'student enjoys reading novels and poetry'
  LCPPN: ['Arts' 'Design']
  LCPN:  ['Arts' 'Design']
  LCPL:  ['Arts' 'Literature']

Query: 'student interested in business analysis'
  LCPPN: ['Arts' 'Design']
  LCPN:  ['Science' 'Biology']
  LCPL:  ['Arts' 'Des

```
🤔 **HICLASS SỬ DỤNG MODEL GÌ BÊN TRONG?**

💡 **TRẢ LỜI NGẮN:**
HiClass KHÔNG có model riêng - nó là WRAPPER sử dụng BẤT KỲ sklearn model nào!

📐 **ARCHITECTURE:**
┌─────────────────────────────────────────┐
│            HiClass (Wrapper)            │
├─────────────────────────────────────────┤
│  ┌─────────────────────────────────────┐ │
│  │    3 HIERARCHICAL PATTERNS:        │ │
│  │  • LCPN (Per Node)                 │ │
│  │  • LCPPN (Per Parent Node)         │ │  
│  │  • LCPL (Per Level)                │ │
│  └─────────────────────────────────────┘ │
├─────────────────────────────────────────┤
│  ┌─────────────────────────────────────┐ │
│  │   USER-CHOSEN BASE CLASSIFIERS:    │ │
│  │  • RandomForestClassifier          │ │
│  │  • LogisticRegression             │ │
│  │  • SVM                            │ │
│  │  • XGBoost                        │ │
│  │  • ANY sklearn-compatible model   │ │
│  └─────────────────────────────────────┘ │
└─────────────────────────────────────────┘
```